In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import math, random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
# from keras.layers import Conv2D, Dense,Flatten, MaxPool2D, Dropout, BatchNormalization, Reshape

print(tf.__version__)

np.random.seed(2019)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_train_file = "../input/Kannada-MNIST/train.csv"
data_test_file = "../input/Kannada-MNIST/test.csv"

df_train = pd.read_csv(data_train_file)
df_test = pd.read_csv(data_test_file)
submissions = pd.read_csv("../input/Kannada-MNIST/sample_submission.csv")

In [ ]:
df_train.head(6)

In [ ]:
df_test.head(8)

In [ ]:
df_train.describe()

In [ ]:
# Note this returns numpy arrays
def get_features_labels(df):
    # The first column is the label.
    labels = df['label'].values
    
    # Select all columns except the first
    features = df.values[:, 1:]/255
    
    return features, labels

In [ ]:
train_features, train_labels = get_features_labels(df_train)
test_features = df_test.values/255
# test_features, test_labels = get_features_labels(df_test)

print(train_features.shape)
print(test_features.shape)
print(train_labels.shape)
print("\n")

test_features1 = np.delete(test_features, 0, 1)
X_train = train_features.reshape(train_features.shape[0], 28, 28, 1)
X_test = test_features1.reshape(test_features1.shape[0], 28, 28, 1)

print(f"Shape of training set is {X_train.shape} now." )
print(f"Shape of test set is {X_test.shape} now." )

In [ ]:
# Used to understand numpy.ndarray. Can be skipped
# b = np.arange(20).reshape(4, 5)
# print(f"Ndarray b is {b}.")
# b_del = np.delete(b, 2, 1)
# print("The remaining array is: ")
# print(b_del)

# c = np.arange(24).reshape(2, 3, 4)
# print(f"Ndarray c is {c}.")
# print("The selected array is: ")
# print(c[:, :, 2])


In [ ]:
# Defaults to showing data from the training set, 
# but we can provide the test data as well, and leave labels as None, to visualize test set
def display_by_index(index, features = X_train, labels = train_labels):
    plt.figure()
    
    if labels is not None:
        plt.title(f'Label: {labels[index]}')
        _ = plt.imshow(features[index, :, :, 0], 'gray')
    else:
        plt.title(f'Sample: {index}')
        _ = plt.imshow(np.reshape(features[index, :], (28,28)), 'gray')
    

In [ ]:
# Visualize a training sample
display_by_index(168)

# Visualize a test sample. Helps to modify parameters of ImageDataGenerator
# This is an example of large rotation
display_by_index(226, features = X_test, labels=None)
# This is an example of large shear
display_by_index(4604, features = X_test, labels=None)
display_by_index(random.randrange(X_test.shape[0]), features = X_test, labels = None)
display_by_index(random.randrange(X_test.shape[0]), features = X_test, labels = None)
display_by_index(random.randrange(X_test.shape[0]), features = X_test, labels = None)
display_by_index(random.randrange(X_test.shape[0]), features = X_test, labels = None)

In [ ]:
df_train['label'].value_counts()
print(df_test.shape)

Y_train_1hot = tf.keras.utils.to_categorical(train_labels)

print(Y_train_1hot.shape)
print("\n")

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train_1hot, random_state = 42, test_size = 0.05)
# X_train = X_train.reshape(X_train.shape[0], 784)
# X_val = X_val.reshape(X_val.shape[0], 784)
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

In [ ]:
model_arch = {}

# lr decay function
# def lr_decay(epoch):
    # return 0.01 * math.pow(0.8, epoch)

# lr schedule callback
# lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', 
                                            patience = 3, # Reduce lr only when accuracy doesn't increase for 3 epochs
                                            verbose = 1, 
                                            factor = 0.3, 
                                            min_lr = 0.00001)

# Plot the decay rate
# x = []
# y = []
# for i in range(1,10):
    # y.append(lr_decay(i))
    # x.append(i)
# plt.plot(y, x)

In [ ]:
# Other models (not used)
# model_arch['single_layer'] = [
      # tf.keras.layers.Input(shape=(28*28,)),
      # tf.keras.layers.Dense(10, activation='softmax')
  # ]

# model_arch['dnn'] = [
      # tf.keras.layers.Input(shape=(28*28,)),
      # tf.keras.layers.Dense(256, activation='sigmoid'),
      # tf.keras.layers.Dense(64, activation='sigmoid'),
      # tf.keras.layers.Dense(10, activation='softmax')
  # ]

# model_arch['dnn_relu'] = [
      # tf.keras.layers.Input(shape=(28*28,)),
      # tf.keras.layers.Dense(256, activation='relu'),
      # tf.keras.layers.Dense(64, activation='relu'),
      # tf.keras.layers.Dense(10, activation='softmax')
  # ]

# Add dropout
# model_arch['dnn_relu_dropout'] = [
      # tf.keras.layers.Input(shape=(28*28,)),
      # tf.keras.layers.Dense(256, activation='relu'),
      # tf.keras.layers.Dropout(0.25),
      # tf.keras.layers.Dense(64, activation='relu'),
      # tf.keras.layers.Dropout(0.25),
      # tf.keras.layers.Dense(10, activation='softmax')
  # ]

In [ ]:
# Generate more images based on training set
generated_data = ImageDataGenerator(
                 rotation_range = 20,
                 shear_range = 0.1,
                 zoom_range = 0.1,
                 width_shift_range = 0.1,
                 height_shift_range = 0.1)

generated_data.fit(X_train)

In [ ]:
# Build convolutional neural network
model_arch['cnn'] = [
      # tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
      tf.keras.layers.Reshape(input_shape = (28, 28, 1), target_shape = (28, 28, 1)),
      tf.keras.layers.Conv2D(filters = 32, kernel_size = 5, activation='relu', padding='same'),
      # tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(filters = 32, kernel_size = 5, activation='relu', padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.MaxPooling2D((2, 2), strides = 1),
      tf.keras.layers.Dropout(0.5),
    
      tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation='relu', padding='same'),
      # tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation='relu', padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.MaxPooling2D((2, 2), strides = 2),
      tf.keras.layers.Dropout(0.5),
    
      tf.keras.layers.Conv2D(filters = 32, kernel_size = 5, activation='relu', padding='same'),
      # tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(filters = 32, kernel_size = 5, activation='relu', padding='same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.MaxPooling2D((2, 2), strides = 2),
      tf.keras.layers.Dropout(0.4),
    
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dropout(0.4),
      tf.keras.layers.Dense(10, activation='softmax')
      ]


In [ ]:
# model_arch.keys()

In [ ]:

model = tf.keras.Sequential(model_arch['cnn'])
# optimizer = 'sgd'
initial_learningrate=2e-3
user_optimizer = RMSprop(lr=initial_learningrate)
# optimizer = 'adam'
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# We will now compile and print out a summary of our model
model.compile(loss = 'categorical_crossentropy',
              optimizer = user_optimizer,
              metrics = ['accuracy'])

model.summary()

In [ ]:
User_batch_size = 256
EPOCHS = 24

In [ ]:
# history = model.fit(train_features, train_labels_1hot, 
          # epochs=EPOCHS, 
          # batch_size=BATCH_SIZE, 
          # validation_split=0.2,
          # callbacks=[lr_decay_callback],
          # shuffle = True)

# Use fit_generator function to train the model on generated images
history = model.fit_generator(generated_data.flow(X_train, Y_train, batch_size = User_batch_size),
                             epochs = EPOCHS, validation_data = (X_val, Y_val), 
                             # steps_per_epoch = X_train.shape[0] // User_batch_size,
                             shuffle = True,  # steps_per_epoch or shuffle, pick one to use
                             verbose = 1, callbacks = [learning_rate_reduction])

In [ ]:
plt.plot(history.history['acc'], color='b', label="Training accuracy")
plt.plot(history.history['val_acc'], color='r', label="Validation accuracy")
plt.legend(loc='lower right', shadow=True)

In [ ]:
# predictions = model.predict_classes(X_test)
predictions = model.predict(X_test)
# submissions = pd.DataFrame({"id": list(range(0,len(predictions))),
                         # "label": predictions})
predictions = np.argmax(predictions, axis = 1)
submissions['label'] = predictions
submissions.to_csv("submission.csv", index = False, header = True)

# submissions.head(8)